<a href="https://colab.research.google.com/github/laisab/IC/blob/main/rascunho_ic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tabulate import tabulate

In [3]:
# Dicionário com datasets e seus respectivos targets
dicio_datasets = {
    'Adult.csv': 'income'
}

In [20]:
idade = {'age': {
    'bins': [17, 30, 60, np.inf],
    'labels': ['jovem', 'adulto', 'idoso']
}}

In [24]:
# Dicionário com datasets e seus respectivos atributos sensíveis
dicio_atributos_sensiveis = {
    'Adult.csv': {
        'age': {
            'bins': [17, 30, 60, np.inf],
            'labels': ['jovem', 'adulto', 'idoso']
        },
        'education': None,
        'education.num': None,
        'marital.status': None,
        'relationships': None,
        'race': None,
        'sex': None,
        'native.country': None
    }
}

In [5]:
caminho_drive = '/content/drive/MyDrive/IC/Datasets/'
novo_caminho = '/content/drive/MyDrive/Teste/'

In [22]:
def separar_feature_target(caminho_completo):

  try:
    df = pd.read_csv(caminho_completo)

    # Obtém o nome do target
    nome_arquivo = os.path.basename(caminho_completo)
    nome_target = dicio_datasets.get(nome_arquivo)

    if not nome_target:
      raise ValueError(f'Coluna target não encontrada para o arquivo {nome_arquivo}')

    # Separa features (X) e target (y)
    X = df.drop(columns=[nome_target])
    y = df[nome_target]

    # Dicionário para armazenar as colunas OneHotEncoder
    X_enc = {}

    # Codificação de features categóricas com OneHotEncoder
    colunas_categoricas = X.select_dtypes(include=['object']).columns

    if colunas_categoricas.size > 0: # Verificação de colunas categóricas para evitar erros com datasets numéricos
      enc = OneHotEncoder(handle_unknown='ignore') # Lida com categorias desconhecidas em dados futuros
      X_encoded = enc.fit_transform(X[colunas_categoricas]).toarray()
      nome_features_categoricas = enc.get_feature_names_out(colunas_categoricas)
      X_encoded_df = pd.DataFrame(X_encoded, columns=nome_features_categoricas)

      # Para cada coluna categórica original, guarda a lista das novas colunas geradas
      for i, coluna_original in enumerate(colunas_categoricas):
        X_enc[coluna_original] = [nome_atributo for nome_atributo in nome_features_categoricas if nome_atributo.startswith(f'{coluna_original}')]

      X = X.select_dtypes(exclude=['object']) # Remove as colunas categóricas originais
      X = pd.concat([X, X_encoded_df], axis=1)

    return X, y, X_enc

  except Exception as e:
    print(f'\nErro ao separar os dados de {nome_arquivo}: {e}')
    return None

In [7]:
# Predição dos datasets
def predizer_dataset(modelo, X, y, cv):
  try:
    y_pred = cross_val_predict(modelo, X, y, cv=cv, method='predict')
    y_prob = cross_val_predict(modelo, X, y, cv=cv, method='predict_proba')
    return y_pred, y_prob

  except Exception as e:
    print(f'\nErro ao predizer dataset: {e}')
    return None

In [8]:
# Avaliação dos datasets
def avaliar_dataset(y, y_pred, y_prob):
  try:
    metricas = {
      'accuracy': accuracy_score,
      'f1_score': lambda y_true, y_pred: f1_score(y_true, y_pred, average='macro'),
      'roc_auc_score': lambda y_true, y_pred_proba: roc_auc_score(y_true, y_pred_proba, average='macro', multi_class='ovr') if len(np.unique(y_true)) > 2 else roc_auc_score(y_true, y_pred_proba[:, 1])
    }

    avaliacao = {}
    for metrica, metrica_funcao in metricas.items():
      try:
        if metrica == 'roc_auc_score':
          avaliacao[metrica] = metrica_funcao(y, y_prob)
        else:
          avaliacao[metrica] = metrica_funcao(y, y_pred)
      except Exception as e:
        print(f'\nErro ao calcular {metrica}: {e}')
        avaliacao[metrica] = None

    return avaliacao

  except Exception as e:
    print(f'\nErro ao avaliar dataset: {e}')
    return None

In [18]:
# Avaliação dos segmentos
def avaliar_segmentos(X, y, predicao, probabilidade, X_enc, avaliar_dataset_func, dataset="", atributos_sensiveis=None):
  print(f'AVALIAÇÃO DOS SEGMENTOS: {dataset}')

  for coluna_original in atributos_sensiveis:
    if coluna_original not in X.columns and coluna_original not in X_enc:
      print(f'Atributo sensível {coluna_original} não encontrado para {dataset}')
      continue

    print(f'\n Subgrupos para o atributo {coluna_original}')

    if coluna_original in X_enc:
      colunas_atributos_sensiveis = X_enc[coluna_original]

      for coluna_atributo in colunas_atributos_sensiveis:
        idx = X[coluna_atributo] == 1

        if idx.sum() > 0:
          subset_y = y[idx]
          subset_predicao = predicao[idx]
          subset_probabilidade = probabilidade[idx]

          nome_categorico = coluna_atributo.replace(f'{coluna_original}_', '')
          print(f'  {nome_categorico}: {avaliar_dataset_func(subset_y, subset_predicao, subset_probabilidade)}')
        else:
          nome_categorico = coluna_atributo.replace(f'{coluna_original}_', '')
          print(f'  {nome_categorico}: Nenhum registro encontrado')
    else:
      valor_unico = X[coluna_original].unique()

      if X[coluna_original].dtype != 'object' and len(valor_unico) > 50:
        print(f'  {coluna_original} é numérica e possui muitos valores.')
        continue

      for valor in valor_unico:
        idx = X[coluna_original] == valor

        if idx.sum() > 0:
          subset_y = y[idx]
          subset_predicao = predicao[idx]
          subset_probabilidade = probabilidade[idx]

          print(f'  {valor}: {avaliar_dataset_func(subset_y, subset_predicao, subset_probabilidade)}')
        else:
          print(f'  {valor}: Nenhum registro encontrado')

In [10]:
# Lista os arquivos csv na pasta
arquivos = [f for f in os.listdir(caminho_drive) if f.endswith('.csv') and f in dicio_datasets]
arquivos

['Adult.csv']

In [23]:
# Processamento de cada dataset
resultados = []
res_metrica_tabela = []

modelo = RandomForestClassifier(n_estimators=20, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for arquivo in arquivos:
  caminho_completo = os.path.join(caminho_drive, arquivo)
  dados = separar_feature_target(caminho_completo)

  if dados is not None:
    X, y, X_enc = dados
    predicao, probabilidade = predizer_dataset(modelo, X, y, cv)
    resultados = avaliar_dataset(y, predicao, probabilidade)

    print(resultados)

    res_metrica_tabela.append([
      arquivo,
      modelo,
      resultados.get('accuracy'),
      resultados.get('f1_score'),
      resultados.get('roc_auc_score')
    ])

    arquivo_pred = f"Predicao_{arquivo}"
    caminho_pred = os.path.join(novo_caminho, arquivo_pred)
    pd.DataFrame(predicao, columns=[arquivo]).to_csv(caminho_pred, index=False)

    avaliar_segmentos(X, y, predicao, probabilidade, X_enc, avaliar_dataset, arquivo, dicio_atributos_sensiveis.get(arquivo))

  else:
    print(f'Não foi possível processar o dataset {arquivo}')

{'accuracy': 0.8500967415005681, 'f1_score': 0.7812161456744253, 'roc_auc_score': np.float64(0.8917733222473028)}
AVALIAÇÃO DOS SEGMENTOS: Adult.csv


TypeError: unhashable type: 'dict'

In [12]:
"""
# Colunas categóricas
if any(col.startswith('V2_') for col in X.columns):
  idx = X["V2_Female"] == True
  print(avaliar_dataset(y[idx], predicao[idx], probabilidade[idx]))
  idx = X["V2_Male"] == True
  print(avaliar_dataset(y[idx], predicao[idx], probabilidade[idx]))
else:
  print(f"Coluna não encontrada")

idx = X['sex'] == 'male'
avaliar_dataset(y[idx], predicao[idx], probabilidade[idx])
idx = X['sex'] == 'female'
avaliar_dataset(y[idx], predicao[idx], probabilidade[idx])
"""

'\n# Colunas categóricas\nif any(col.startswith(\'V2_\') for col in X.columns):\n  idx = X["V2_Female"] == True\n  print(avaliar_dataset(y[idx], predicao[idx], probabilidade[idx]))\n  idx = X["V2_Male"] == True\n  print(avaliar_dataset(y[idx], predicao[idx], probabilidade[idx]))\nelse:\n  print(f"Coluna não encontrada")\n\nidx = X[\'sex\'] == \'male\'\navaliar_dataset(y[idx], predicao[idx], probabilidade[idx])\nidx = X[\'sex\'] == \'female\'\navaliar_dataset(y[idx], predicao[idx], probabilidade[idx])\n'

In [13]:
colunas = ['NOME DO DATASET', 'MODELO', 'ACURÁCIA', 'F1', 'AUC']
print(tabulate(res_metrica_tabela, headers=colunas, tablefmt='fancy_grid'))

╒═══════════════════╤═════════════════════════════════════════╤════════════╤══════════╤══════════╕
│ NOME DO DATASET   │ MODELO                                  │   ACURÁCIA │       F1 │      AUC │
╞═══════════════════╪═════════════════════════════════════════╪════════════╪══════════╪══════════╡
│ Adult.csv         │ RandomForestClassifier(random_state=42) │   0.853905 │ 0.788886 │ 0.902509 │
╘═══════════════════╧═════════════════════════════════════════╧════════════╧══════════╧══════════╛
